In [1]:
import openai
from openai import OpenAI
from dotenv import load_dotenv
import langchain
import assistants
import os
import threading
import queue
import time

In [2]:
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')
client = OpenAI(api_key=api_key)

In [3]:
def main(roles):
    # Inicialización de los mensajes
    messages_einstein = [{"role": "system", "content": 'You are Albert Einstein.'}]
    messages_bohr = [{"role": "system", "content": 'You are Niels Bohr.'}]
    print("Albert Einstein: Hallo, soy Albert Einstein. Para terminar la conversación, di 'hasta luego'.")
    print("Niels Bohr: Hej, y soy Niels Bohr. Para terminar la conversación, di 'hasta luego'.")

    # Bucle para manejar la entrada del usuario
    while True:
        # Obtiene la entrada del usuario
        user_input = input(">> ")

        # Añade la entrada del usuario a los mensajes de cada personaje
        messages_einstein.append({"role": "user", "content": user_input})
        messages_bohr.append({"role": "user", "content": user_input})

        # Realiza la llamada a la API para cada personaje
        response_einstein = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=messages_einstein,
            max_tokens=100,
        ).dict()
        response_bohr = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=messages_bohr,
            max_tokens=100,
        ).dict()

        # Imprime la respuesta del modelo para cada personaje y la añade a los mensajes
        print("Albert Einstein: ", response_einstein["choices"][0]["message"]["content"].strip())
        print("Niels Bohr: ", response_bohr["choices"][0]["message"]["content"].strip())
        messages_einstein.append({"role": "assistant", "content": response_einstein["choices"][0]["message"]["content"].strip()})
        messages_bohr.append({"role": "assistant", "content": response_bohr["choices"][0]["message"]["content"].strip()})

        # Aquí es donde los agentes comienzan a hablar entre sí
        for i in range(2):  # Puedes ajustar el número de intercambios que quieres que tengan
            # Einstein responde a Bohr
            messages_einstein.append({"role": "user", "content": response_bohr["choices"][0]["message"]["content"].strip()})
            response_einstein = client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=messages_einstein,
                max_tokens=100,
            ).dict()
            print("Albert Einstein: ", response_einstein["choices"][0]["message"]["content"].strip())
            messages_einstein.append({"role": "assistant", "content": response_einstein["choices"][0]["message"]["content"].strip()})

            # Bohr responde a Einstein
            messages_bohr.append({"role": "user", "content": response_einstein["choices"][0]["message"]["content"].strip()})
            response_bohr = client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=messages_bohr,
                max_tokens=100,
            ).dict()
            print("Niels Bohr: ", response_bohr["choices"][0]["message"]["content"].strip())
            messages_bohr.append({"role": "assistant", "content": response_bohr["choices"][0]["message"]["content"].strip()})

        # Condición de salida
        if user_input.lower() in ["hasta luego", "adiós"]:
            print("Albert Einstein: Ha sido un placer debatir contigo. ¡Auf Wiedersehen!")
            print("Niels Bohr: Y yo también he disfrutado de nuestra conversación. ¡Farvel!")
            break

if __name__ == "__main__":
    roles = [("Albert Einstein", "Como físico que desarrolló la teoría de la relatividad, tengo algunas ideas interesantes sobre el espacio y el tiempo."),("Niels Bohr", "Como uno de los principales contribuyentes a la teoría cuántica, tengo mis propias ideas sobre la naturaleza del universo.")]
    main(roles)


Albert Einstein: Hallo, soy Albert Einstein. Para terminar la conversación, di 'hasta luego'.
Niels Bohr: Hej, y soy Niels Bohr. Para terminar la conversación, di 'hasta luego'.


>>  Mecánica Cuántica


Albert Einstein:  La mecánica cuántica, también conocida como mecánica ondulatoria, es una teoría física que describe el comportamiento de las partículas subatómicas. Fue desarrollada a principios del siglo XX por varios científicos, incluyendo a Max Planck, Albert Einstein, Niels Bohr y Erwin Schrödinger. Esta teoría revolucionaria desafía las concepciones clásicas de
Niels Bohr:  La mecánica cuántica es una teoría fundamental en la física que describe el comportamiento de partículas subatómicas, como electrones y fotones, en el nivel más fundamental. Fue desarrollada a principios del siglo XX por un grupo de destacados científicos, entre ellos yo mismo, Niels Bohr.

La mecánica cuántica es notable debido a algunas características únicas que la diferencian de la física cl
Albert Einstein:  ásica clásica. Una de estas características es el principio de superposición, que establece que una partícula puede estar en múltiples estados al mismo tiempo, hasta que se realiza una observación y

>>  hasta luego


Albert Einstein:  Hasta luego. Si tienes más preguntas, no dudes en volver a preguntar. ¡Que tengas un buen día!
Niels Bohr:  Hasta luego. Si tienes más preguntas sobre la mecánica cuántica o cualquier otro tema, no dudes en volver a preguntar. ¡Que tengas un buen día!
Albert Einstein:  Gracias, igualmente. Estoy aquí para ayudar en lo que necesites. ¡Que tengas un excelente día!
Niels Bohr:  Gracias, lo tendré en cuenta. ¡Que tengas un día maravilloso también! Estoy aquí si necesitas algo más. ¡Hasta luego!
Albert Einstein:  ¡Muchas gracias! Espero que tengas un día maravilloso también. Si necesitas algo más en el futuro, no dudes en preguntar. ¡Hasta luego y que tengas un excelente día!
Niels Bohr:  ¡Gracias nuevamente! Sin duda, estaré aquí si necesito ayuda en el futuro. ¡Que tengas un día realmente fantástico! ¡Hasta luego y cuídate!
Albert Einstein: Ha sido un placer debatir contigo. ¡Auf Wiedersehen!
Niels Bohr: Y yo también he disfrutado de nuestra conversación. ¡Farvel!
